# The GRAD-GPAD framework 🗿 
    
    ➡️ Foundations: The Python Library
---

The `gradgpad` framework povide python modules to ease face-PAD research. This tutorial is designed to help you become familiar with the package, undertanding the **Foundations** of the package.


![gradgpad_detailed_architecture](images/gradgpad_detailed_architecture.jpeg "GRAD-GPAD Architecture")



## Table of Contents 👩‍💻
- Installation 💻
- Environment
- Annotations
- Scores
- Results
- Metrics 

### Installation 💻

In [ ]:
pip install -U gradgpad

---
### Environment

We start importing required objects:

In [ ]:
import json
from gradgpad import (
    annotations,
    ScoresProvider,
    Approach,
    Protocol,
    Subset,
    Dataset,
    Device,
    CoarseGrainedPai,
    GrainedPaiMode,
    Demographic,
    Filter,
    Sex,
    Age,
    SkinTone,
    Scenario,
    Metrics,
    MetricsDemographics,
    ResultsProvider
)

---
### Annotations

There are nearly 30000 annotations for the 13 datasets presented in the GRAD-GPAD v2.

In [ ]:
assert annotations.num_annotations == 29567

You can print each of the annotations using sorted index. Note if not select an annotation index it will print the `29567` annotations, but this is not recommended to use on a Notebook.

In [ ]:
print(annotations.annotated_samples[0])
annotations.print_semantic(annotation_index=0)

You can also, filter by dataset ids as the example below:

In [ ]:
selected_ids = [
    "replay-mobile_devel/real/client023_session02_authenticate_tablet_controlled.mov", 
    "replay-mobile_devel/real/client025_session02_authenticate_tablet_controlled.mov",
    "replay-mobile_devel/real/client029_session02_authenticate_mobile_direct.mov",
    "replay-mobile_devel/real/client005_session02_authenticate_mobile_controlled.mov"]
selected_annotations = annotations.get_annotations_from_ids(selected_ids)
assert len(selected_annotations) == len(selected_ids)

Obtain statistics from `annotations` with `statistics` function.

In [ ]:
statistics = annotations.statistics()
print(json.dumps(statistics, indent=4, sort_keys=True))

This stadistic are reflected on work [CITATION] Figure XXXX

<img src="images/grad-gpad-pais.png" width="300">

---
### Scores

The `gradgpad` library has available a class to provide scores (`ResultProvider`).
To retrieve all scores for a specific approach, use `ScoresProvider.all(approach)`

In [ ]:
scores_quality = ScoresProvider.all(Approach.QUALITY_RBF)
scores_auxiliary = ScoresProvider.all(Approach.AUXILIARY)

To retrieve all scores by subsets, use `ScoresProvider.get_subsets(approach, protocol)`:

In [ ]:
scores_auxiliary_subsets = ScoresProvider.get_subsets(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST
)
scores_auxiliary_devel = scores_auxiliary_subsets.get("devel")
scores_auxiliary_test = scores_auxiliary_subsets.get("test")

Besides, you can obtain scores with `ScoresProvider.get(approach, protocol, subset, dataset, device, pai)`

In [ ]:
scores_auxiliary_test = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST)

Or more specific scores

In [ ]:
scores_specific = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST, 
    Dataset.REPLAY_MOBILE, 
    Device.MOBILE_TABLET, 
    CoarseGrainedPai.REPLAY
)

You can check every option with static method `options`

In [ ]:
print(f"- Approaches:\n  > {Approach.options()}\n")
print(f"- Protocol:\n  > {Protocol.options()}\n")
print(f"- Subset:\n  > {Subset.options()}\n")
print(f"- Dataset:\n  > {Dataset.options()}\n")
print(f"- CoarseGrainedPai:\n  > {CoarseGrainedPai.options()}\n")

Additionaly, provided `Scores` class implements useful tools to manage obtained scores from an experiment: 

In [ ]:
scores = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST
)

genuine_scores = scores.get_genuine()
genuine_attacks = scores.get_attacks()

Using `numpy` for data management in your experiments, just use `get_numpy_scores` and `get_numpy_labels`

In [ ]:
numpy_scores_genuine = scores.get_numpy_scores()
numpy_scores_labels = scores.get_numpy_labels()

Another interesting tool that implements `Scores` object is the filtering method `filtered_by` wich uses `Filter` object.

Imagine we want filter by `Sex`, `Age` or `SkinTone`:

In [ ]:
# Sex
male_scores = scores.filtered_by(Filter(sex=Sex.MALE))
female_scores = scores.filtered_by(Filter(sex=Sex.FEMALE))

# Age
young_scores = scores.filtered_by(Filter(age=Age.YOUNG))
adult_scores = scores.filtered_by(Filter(age=Age.ADULT))
senior_scores = scores.filtered_by(Filter(age=Age.SENIOR))

# SkinTone
light_pink_scores = scores.filtered_by(Filter(skin_tone=SkinTone.LIGHT_PINK))
light_yellow_scores = scores.filtered_by(Filter(skin_tone=SkinTone.LIGHT_YELLOW))
medium_pink_brown_scores = scores.filtered_by(Filter(skin_tone=SkinTone.MEDIUM_PINK_BROWN))
medium_dark_brown_scores = scores.filtered_by(Filter(skin_tone=SkinTone.MEDIUM_DARK_BROWN))
dark_brown_scores = scores.filtered_by(Filter(skin_tone=SkinTone.DARK_BROWN))

Or just filter by dataset

In [ ]:
casia_fasd_scores = scores.filtered_by(Filter(dataset=Dataset.CASIA_FASD))
threedmad_scores = scores.filtered_by(Filter(dataset=Dataset.THREEDMAD))
uvad_scores = scores.filtered_by(Filter(dataset=Dataset.UVAD))
siw_m_scores = scores.filtered_by(Filter(dataset=Dataset.SIW_M))
siw_scores = scores.filtered_by(Filter(dataset=Dataset.SIW))
rose_youtu_scores = scores.filtered_by(Filter(dataset=Dataset.ROSE_YOUTU))
replay_mobile_scores = scores.filtered_by(Filter(dataset=Dataset.REPLAY_MOBILE))
replay_attack_scores = scores.filtered_by(Filter(dataset=Dataset.REPLAY_ATTACK))
oulu_npu_scores = scores.filtered_by(Filter(dataset=Dataset.OULU_NPU))
msu_mfsd_scores = scores.filtered_by(Filter(dataset=Dataset.MSU_MFSD))
hkbu_v2_scores = scores.filtered_by(Filter(dataset=Dataset.HKBU_V2))
hkbu_scores = scores.filtered_by(Filter(dataset=Dataset.HKBU))
csmad_scores = scores.filtered_by(Filter(dataset=Dataset.CSMAD))

Or by scenario:
* **Genuine** (`Scenario.GENUINE`): Genuine attemps;
* **PAS I** (`Scenario.PAS_TYPE_I`): represents a scenario where spoofers have freedom to try to impersonate an identity completely (as with a stolen cell phone or in an isolated access environment);
* **PAS II** (`Scenario.PAS_TYPE_II`): represents partial identity impersonations, where attackers only use a part of someone else’s identity;
* **PAS III** (`Scenario.PAS_TYPE_III`): where users try to hide their identity without impersonating anyone in particular

In [ ]:
genuine_scores = scores.filtered_by(Filter(scenario=Scenario.GENUINE))
pas_I_scores = scores.filtered_by(Filter(scenario=Scenario.PAS_TYPE_I))
pas_II_scores = scores.filtered_by(Filter(scenario=Scenario.PAS_TYPE_II))
pas_III_scores = scores.filtered_by(Filter(scenario=Scenario.PAS_TYPE_III))

Finally, `Scores` class provides you fair subsets (balanced) for demographic labels.

In [ ]:
sex_fair_subset = scores.get_fair_sex_subset()
age_fair_subset = scores.get_fair_age_subset()
skin_tone_fair_subset = scores.get_fair_skin_tone_subset()

---
### Metrics

There are three ways of calculating metrics using the `gradgpad` `Python` package: using the `Metrics`
and `MetricsDemographic` classes and/or using the available functions on module `gradgpad.foundations.metrics`. 

First, if we use the `Metrics` class, we can automatize the calculation of basic Metrics (e.g *EER*): Besides, it
provides a method to calculate an in-depth analysis taking into account fine and coarse grained PAI categorization.

In [ ]:
scores_auxiliary_devel = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.DEVEL
)

scores_auxiliary_test = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST
)

metrics = Metrics(
    devel_scores=scores_auxiliary_devel,
    test_scores=scores_auxiliary_test
)

# Obtain Equal Error Rates values for DEVEL and TEST subsets
eer_devel = metrics.get_eer(Subset.DEVEL)
eer_test = metrics.get_eer(Subset.TEST)
eer_devel_threshold = metrics.get_eer_th(Subset.DEVEL)
eer_test_threshold = metrics.get_eer_th(Subset.TEST)

# Obtain an in-depth analysis from the values of DEVEL and TEST subsets 
# fixing the working points for BPCER and APCER ([0-1]).
bpcer_fixing_working_points = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.20
]  
apcer_fixing_working_points = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.20,
]  
indepth_analysis = metrics.get_indepth_analysis(
    bpcer_fixing_working_points, 
    apcer_fixing_working_points,
    GrainedPaiMode.options()
)

In [ ]:
print(f"AUXILIARY EER [Devel: {metrics.get_eer(Subset.DEVEL)} | Test: {metrics.get_eer(Subset.TEST)}]")
print(f"AUXILIARY EER Th [Devel: {metrics.get_eer_th(Subset.DEVEL)} | Test: {metrics.get_eer_th(Subset.TEST)}]")

In [ ]:
print(json.dumps(indepth_analysis, indent=4, sort_keys=True))

This contains many detailed metrics for both fine-grained and coarse-grained PAI calculation. 

On the other hand, if we want to calculate specific metrics for usability on demographic groups, the package provdes the `MetricsDemographic` class.

In [ ]:
metrics_demographics = MetricsDemographics(
    devel_scores=scores_auxiliary_devel,
    test_scores=scores_auxiliary_test
)

bpcer_sex = metrics_demographics.get_bpcer_sex()
bpcer_age = metrics_demographics.get_bpcer_age()
bpcer_skin_tone = metrics_demographics.get_bpcer_skin_tone()


print(json.dumps(bpcer_sex, indent=4, sort_keys=True))
print(json.dumps(bpcer_age, indent=4, sort_keys=True))
print(json.dumps(bpcer_skin_tone, indent=4, sort_keys=True))

Finally, functions provided in the module `gradgpad.fountations.metrics` can be used to calculate specific metrics as in the following example:

In [ ]:
from gradgpad.foundations.metrics.eer import eer
from gradgpad.foundations.metrics.far import far
from gradgpad.foundations.metrics.frr import frr

devel_scores = scores_auxiliary_devel.get_numpy_scores()
devel_labels = scores_auxiliary_devel.get_numpy_labels()

eer_value, eer_threshold = eer(devel_scores, devel_labels)
far_01_value, far_01_threshold = far(devel_scores, devel_labels, 0.01)
ffr_05_value, frr_05_threshold = frr(devel_scores, devel_labels, 0.05)

In [ ]:
print(f"AUXILIARY EER [Devel: {eer_value*100:.2f} % (threshold {eer_threshold:.2f})]")
print(f"AUXILIARY FAR @ FRR=1% [Devel: {far_01_value*100:.2f} % (threshold {far_01_threshold:.2f})]")
print(f"AUXILIARY FRR @ FAR=5% [Devel: {ffr_05_value*100:.2f} % (threshold {frr_05_threshold:.2f})]")

Some metrics needs to select a working point in one subset (e.g `DEVEL`) and calculate the metric in another (e.g `TEST`). For example, the `HTER` (*Half Total Error Rate*).

In [ ]:
from gradgpad.foundations.metrics.hter import hter


test_scores = scores_auxiliary_test.get_numpy_scores()
test_labels = scores_auxiliary_test.get_numpy_labels()

hter_value = hter(test_scores, test_labels, eer_threshold)

print(f"HTER: {hter_value*100:.2f} %")

For `BPCER`(*Bonafide Presentation Classifier Error Rate*), `APCER` (*Attack Presentation Classification Error Rate*) and `ACER` (*Average Classification Error Rate*) we have the following functions available.

In [ ]:
from gradgpad.foundations.metrics.apcer import apcer
from gradgpad.foundations.metrics.bpcer import bpcer
from gradgpad.foundations.metrics.acer import acer

apcer_value = apcer(test_scores, test_labels, eer_threshold)
bpcer_value = bpcer(test_scores, test_labels, eer_threshold)
acer_value = acer(test_scores, test_labels, eer_threshold)

print(f"APCER: {apcer_value*100:.2f} %")
print(f"BPCER: {bpcer_value*100:.2f} %")
print(f"ACER: {acer_value*100:.2f} %")

The `gradgpad` package also has available function to obtain both `APCER` and `BPCER` fixing the opposite. In this way, we can evaluate the security of the system for an specific usability (e.g $APCER @ BPCER$) or viceversa.

In [ ]:
from gradgpad.foundations.metrics.apcer_fixing_bpcer import apcer_fixing_bpcer

bpcer_10 = 0.1
apcer_value_bpcer_10 = apcer_fixing_bpcer(test_scores, test_labels, bpcer_10)
bpcer_20 = 0.2
apcer_value_bpcer_20 = apcer_fixing_bpcer(test_scores, test_labels, bpcer_20)

print(f"APCER @ BPCER=10%: {apcer_value_bpcer_10*100:.2f} %")
print(f"APCER @ BPCER=20%: {apcer_value_bpcer_20*100:.2f} %")


Otherwise, if we fix the security point, we can evaluate the usability ($APCER @ BPCER$)

In [ ]:
from gradgpad.foundations.metrics.bpcer_fixing_apcer import bpcer_fixing_apcer

apcer_10 = 0.1
bpcer_value_bpcer_10 = bpcer_fixing_apcer(test_scores, test_labels, apcer_10)
apcer_20 = 0.2
bpcer_value_bpcer_20 = bpcer_fixing_apcer(test_scores, test_labels, apcer_20)

print(f"BPCER @ APCER=10%: {bpcer_value_bpcer_10*100:.2f} %")
print(f"BPCER @ APCER=20%: {bpcer_value_bpcer_20*100:.2f} %")

---
### Results

`ResultsProvider` is implemented on top on scores and provides a high level of abstraction to obtain results from Scores and Metric calculators.

In [ ]:
results_auxiliary_grandtest = ResultsProvider.grandtest(Approach.AUXILIARY)
results_auxiliary_cross_dataset = ResultsProvider.cross_dataset(Approach.AUXILIARY)
results_auxiliary_lodo = ResultsProvider.lodo(Approach.AUXILIARY)
results_auxiliary_cross_device = ResultsProvider.cross_device(Approach.AUXILIARY)
results_auxiliary_unseen_attack = ResultsProvider.unseen_attack(Approach.AUXILIARY)

If you print the result dictionary, you can check detailed result report. This function loads the scores and calculate several working points for default face-PAD metrics.

In [ ]:
print(json.dumps(results_auxiliary_grandtest, indent=4, sort_keys=True))

You also can use `ResultsProvider.all` to calculate each available protocol.

In [ ]:
results_auxiliary = ResultsProvider.all(Approach.AUXILIARY)

Additionally, `ResultsProvider` can calculate results for evenly demographic distributed subsets for the Grandtest Protocol:

In [ ]:
results_auxiliary_grandtest_sex = ResultsProvider.grandtest_fair_demographic_bpcer(
    Approach.AUXILIARY, Demographic.SEX
)
results_auxiliary_grandtest_age = ResultsProvider.grandtest_fair_demographic_bpcer(
    Approach.AUXILIARY, Demographic.AGE
)
results_auxiliary_grandtest_skin_tone = ResultsProvider.grandtest_fair_demographic_bpcer(
    Approach.AUXILIARY, Demographic.SKIN_TONE
)

print(json.dumps(results_auxiliary_grandtest_sex, indent=4, sort_keys=True))
print(json.dumps(results_auxiliary_grandtest_age, indent=4, sort_keys=True))
print(json.dumps(results_auxiliary_grandtest_skin_tone, indent=4, sort_keys=True))